# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from the Enron email dataset in a MongoDB database, and write a report detailing the suitability of different types of databases for data science applications.  You will be required to run code to answer the given questions in the Jupyter notebook provided, and write a report describing alternative approaches to using MongoDB.

Download the JSON version of the Enron data and import into a collection called messages in a database called enron.  You do not need to set up any authentication.  In the Jupyter notebook provided, perform the following tasks, using the Python PyMongo library.  Marks available for each question are enclosed in square brackets.

Answers should be efficient in terms of speed.  Answers which are less efficient will not get full marks.

In [1]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint
import re
import pandas as pd

### 1)
Write a function which returns a MongoDB connection object to the "messages" collection. **[1]**

In [2]:
#function to return a MongoDB connection object

def get_collection():
    
    client = MongoClient('mongodb://localhost:27017') #connecting to localhost, listening on port 27017
    print ("Connected successfully!!!")
    
    client.database_names()         #to return a list of all current databases
    
    db = client.enron               #database object created
    
    myCol=db.messages               #collection created
    return myCol

#end of function get_collection()

messages=get_collection()   #call to function get_collection()
messages                    #to display the collection

Connected successfully!!!


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'enron'), 'messages')

In [ ]:
%%bash
#this part of the code is not asked in the questions, but it is executed to import the enron data, and so, it's executed only once

wget http://svm-hf1g10-data-science.ecs.soton.ac.uk/messages.json.gz       #to download the data
gunzip messages.json.gz          #to unzip the file
mongoimport --db enron --collection messages --drop --file messages.json   #to import data
ls -lh *.json                   #to check if the json file is present in the specified folder or not                                       

### 2)

Write a function which returns the amount of emails in the messages collection in total **[1]**

In [3]:
#function to return the amount of emails in the messages collection

def get_amount_of_messages(collection):
    
    msgCount=collection.count()
    return msgCount

#end of function get_amount_of_messages()

messagesCount=get_amount_of_messages(messages)   #call to function get_amount_of_messages()
messagesCount        #to display the count

501513

### 3) 

Write a function which returns each person who was BCCed on an email.  Include each person only once, and display only their name according to the X-From header. **[2]**



In [4]:
#function to return each person BCCed on an email

def get_bcced_people(collection):
    
    filter={'headers.X-bcc':{'$exists':'true', '$regex':'\S'}}    #to check if there's at least one entry in 'X-Bcc' field
    cursor = collection.find(filter)                             
    
    bccPeople = set()           #a set is created to avoid storing repeated elements
    
    for c in cursor:
        bccPeople.add(c['headers']['X-From'])     #for each element in the cursor,the X-From values are stored in the set
    
    bccNewList=str(bccPeople)                #each value is converted into a string in order to use regular expression.
    return re.sub(r'<.*?>','',bccNewList)    #to eliminate the extra characters and retain the names only

# end of function get_bcced_people()

bccPpl = get_bcced_people(messages)     #call to function get_bcced_people()
print(bccPpl)             #to display the names of BCCed people

{'Gray, Barbara N. ', 'Kevin Ruscitti', 'Forney, John M. ', 'Scott, Susan M. ', 'Robertson, Audrey ', 'Heard, Marie ', 'Jeff Dasovich', 'Jeff Skilling', 'Vince J Kaminski', 'Lu, Zimin', 'Schwieger, Jim ', 'Sanders, Richard B. ', 'Carr, James ', 'Lay, Kenneth ', 'Dorland, Chris ', 'Smith, Gary ', 'Kevin M Presto', 'Thomas, Sheri ', 'Williamson, Joannie ', 'Stacey W White', 'Kean, Steven J. ', 'Fitzgerald, Genia ', 'Dasovich, Jeff ', 'V Charles Weldon', 'Gerald Nemec', 'Beck, Sally ', 'Hayslett, Rod ', 'Perlingiere, Debra ', 'Taylor, Mark E (Legal) ', 'Geaccone, Tracy ', 'Teal, Donna ', 'Mendez, Nicole ', 'Germany, Chris ', 'Kitchen, Louise ', 'Wilkens, Jerry ', 'Davis, Dana ', 'Denne, Karen ', 'Valdez, Christina ', 'White, Stacey W. ', 'Shapiro, Richard ', 'Umanoff, Adam ', 'Charles, Constance ', 'Shackleton, Sara ', 'Thompson, Patti ', 'Corman, Shelley ', 'Whalley, Greg ', 'Sally Beck', 'Matthews, Ron ', 'Ring, Richard ', 'Hyatt, Kevin ', 'Gilbert-smith, Doug ', 'John J Lavorato', 'Weh

### 4)
Write a function with parameter subject, which gets all emails in a thread with that parameter, and orders them by date(ascending) **[3]**

In [5]:
#function to get ell emails is a thread defined by the 'subject' parameter

def get_emails_in_thread(collection, subject):
    
    filter = {'headers.Subject' : subject}       #to filter the emails on the specified subjects
    cursor = collection.find(filter)
    
    newList = []

    #to handle the time format for sorting
    for doc in cursor:
        newDoc = doc
        if (doc['headers']['Date'][-5:] == '(PST)'):      #to format Date according to strptime format for PST/PDT timezones
            newDoc['headers']['Date'] = datetime.strptime(doc['headers']['Date'], '%a, %d %b %Y %H:%M:%S %z (PST)')
        else:
            newDoc['headers']['Date'] = datetime.strptime(doc['headers']['Date'], '%a, %d %b %Y %H:%M:%S %z (PDT)')
            
        newList.append(newDoc)
    
    finalList = sorted(newList, key=lambda k: k['headers']['Date'])   #sorting based on the converted, new time format
    
    #converting Date field back to its original format for the purpose of displaying
    for finalDoc in finalList:
        finalDoc['headers']['Date'] = datetime.strftime(finalDoc['headers']['Date'], '%a, %d %b %Y %H:%M:%S %z (PST)')
    
    return finalList

#end of function get_emails_in_thread()

sub='brown bags'     #example subject to check the emails in this thread
finallist=get_emails_in_thread(messages, sub)     #call to function get_emails_in_thread

for docs in finallist:
    pprint(docs)

{'_id': ObjectId('4f16fc97d1e2d32371003e63'),
 'body': 'Have you signed up?\n'
         '\n'
         'Jen Fraser, a Director with Enron Global Markets, November 14 from '
         '12 to 1 in \n'
         'EB 30C\n'
         '\n'
         '\n'
         'Call or email Stacey Dempsey.\n',
 'filename': '504.',
 'headers': {'Content-Transfer-Encoding': '7bit',
             'Content-Type': 'text/plain; charset=us-ascii',
             'Date': 'Mon, 13 Nov 2000 00:22:00 -0800 (PST)',
             'From': 'stacey.dempsey@enron.com',
             'Message-ID': '<6708226.1075854679184.JavaMail.evans@thyme>',
             'Mime-Version': '1.0',
             'Subject': 'brown bags',
             'To': 'craig.hall@enron.com, ryan.hinze@enron.com, '
                   'jennifer.martinez@enron.com, \r\n'
                   '\tjames.whitehead@enron.com, darren.maloney@enron.com, \r\n'
                   '\tcrystal.hyde@enron.com, benjamin.markey@enron.com, \r\n'
                   '\tgeorge.thomas@en

                   '\tkarla.compean@enron.com, zal.masani@enron.com, \r\n'
                   '\tleonard.tham@enron.com, claire.broido@enron.com, \r\n'
                   '\tyvan.chaxel@enron.com, pearce.hammond@enron.com, \r\n'
                   '\telizabeth.howley@enron.com, usman.shaukat@enron.com, \r\n'
                   '\tmisti.day@enron.com, simone.rose@enron.com, '
                   'michelle.zhang@enron.com, \r\n'
                   '\tshilpa.chunchu@enron.com, daniel.graham@enron.com, \r\n'
                   '\tmolly.hellerman@enron.com, david.hunker@enron.com, \r\n'
                   '\talberto.jimenez@enron.com, jebong.lee@enron.com, \r\n'
                   '\tchetan.paipanandiker@enron.com, '
                   'eva.rainer@enron.com, \r\n'
                   '\tcarlos.ruiz@enron.com, stephanie.segura@enron.com, \r\n'
                   '\trahul.seksaria@enron.com, michael.simmons@enron.com, \r\n'
                   '\timad.tareen@enron.com, todd.roten@enron.com, '
  

### 5)

Write a function which returns the percentage of emails sent on a weekend (i.e., Saturday and Sunday) as a `float` between 0 and 1 **[3]**

In [6]:
#function to return the percentage of emails "Sent" on a weekend

def get_percentage_sent_on_weekend(collection):  
    
    totalMails=collection.find().count()
    
    #to filter the emails sent on weekend, and also to check if the mails were actually sent, and not just saved as drafts
    filter = {
        "$and":[{'headers.Date' : {'$regex' : '^(Sat|Sun)'}}, {'headers.To' : {'$exists' : 'true'}}]
         }
    weekendMails = collection.find(filter).count()
    
    percentage=weekendMails/totalMails     #ratio of emails sent on weekends to the total number of emails
    return percentage
#end of function get_percentage_sent_on_weekend()

per=get_percentage_sent_on_weekend(messages)     #call to function get_percentage_sent_on_weekend()
per      #to display the percentage of emails

0.038124634854928986

### 6)

Write a function which takes an argument `email_address`, and `limit`.  The function should return the amount of emails sent, received, and total (sent + received) between each other email address, and the email address specified as an argument, as follows: `[{"contact": "michael.simmons@enron.com", "from": 42, "to": 92, "total": 134}, {"contact"......}]`. Use the `To`, `From`, and `Cc` headers.
* Sort the output by the total amount of emails descending
* The parameter limit which specifies how many results should be returned.  If it is null, the function should return them all. **[5]**

In [7]:
#function to return the total amount of emails sent/recieved between "email_address" and every other email address
def get_emails_between_contacts(collection, email_address, limit):
  #MATCHES
  #match 1 is done to find the emails sent by "email_address" and match2 is done to find the emails recieved by "email_address"

    match1 = {            #to find the emails whose "From" field is the same as "email_address" specified in the argument
            '$match': {'headers.From': email_address}
        }

    match2 = {          #to find the emails whose "To" and "Cc" fields are same as "email_address" specified in the argument
           '$match': { '$or':[{'headers.To': email_address},{'headers.Cc' : email_address}]}
        }

   #GROUPS
    group1 = {               #to group the all emails sent by the "email_address" to someone
            '$group': {'_id': '$headers.To' , 'to': {'$sum': 1}}
        }
    
    group2 = {             #to group the all emails received by the "email_address" from someone
            '$group': {'_id': '$headers.From' , 'from': {'$sum': 1}}
        }
    
    group3 = {            #for cases where the email was sent by "email_address" but there was no reply
            '$group': {'_id': '$headers.To'}
        }
    
   #SORTING
    sort1 = {           #sorting of the count of "to" emails in descending order
            '$sort': {'to': pymongo.DESCENDING}
        }
    
    sort2 = {           #sorting of the count of "from" emails in descending order
            '$sort': {'from': pymongo.DESCENDING}
        }
    
    cursor1 = messages.aggregate([match1, group1, sort1])    #for grouping and sorting all emails sent by "email_address"
    cursor2 = messages.aggregate([match2, group2, sort2])    #for grouping and sorting all emails recieved by "email_address"
    cursor3=messages.aggregate([match1,group3])      #for grouping if there was a mail sent by "email_address" but never recieved replies

   #for iterating over both cursors
    list1=[c1 for c1 in cursor1]
    list2=[c2 for c2 in cursor2]

    pdList1 = pd.DataFrame(list1)      #creating a data frame using pandas with the first list (for first case)
    emails1 = list(pdList1._id.unique())     #to avoid repitions of document ids in the list

    pdList2 = pd.DataFrame(list2)      #creating a data frame using pandas with the second list (for second case)
    emails2 = list(pdList2._id.unique())    #to avoid repitions pf document ids in the list

    set(emails1+emails2)    #to create unordered collection of unique elements

    #merging the dataframes to align the records from each, based on common attributes
    contactDoc=pd.merge(pdList1, pdList2, on="_id", how='outer')   
    
    #for all the NaN values in the dataframe, conversion into 0 is necessary for arithmetic calculations
    contactDoc=contactDoc.fillna(0)
    
    contactDoc=contactDoc[contactDoc._id !=0]       #to check if the id exists
    
    #converting all the column values into integer values because the data frames initially has floating point numbers
    contactDoc['to']=contactDoc['to'].astype(int)
    contactDoc['from']=contactDoc['from'].astype(int)
    contactDoc['total']=contactDoc['to']+contactDoc['from']     #for adding the "to" and "from" emails
    
    #to sort the emails based on the total number of emails sent/received
    contactDoc=contactDoc.sort_values(by=['total'],ascending=False)
    
    #for displaying the whole list if limit is specified as None
    if limit!=None:
        contactDoc=contactDoc.head(limit)           #to pick the top emails, the number of which is specified by "limit"
    
    contactDoc = contactDoc.rename(columns={'_id': 'Contact'})    #converting field name into "contact" as specified in the requirement
    return contactDoc.to_dict('records')        #converting the data frame into a python dictionary
#end of function get_emails_between_contacts()


#example to check the above function
email_addr='jason.bass2@compaq.com'           
contactSheet=get_emails_between_contacts(messages, email_addr, 5)   #call to function get_emails_between_contacts() with limit=5
contactSheet

[{'Contact': 'eric.bass@enron.com', 'from': 111, 'to': 5, 'total': 116},
 {'Contact': 'dfranklin@hanovermeasurement.com, eric.bass@enron.com, \r\n\tdaphneco64@bigplanet.com, lwbthemarine@bigplanet.com, \r\n\tlqcolombo@aol.com',
  'from': 0,
  'to': 1,
  'total': 1},
 {'Contact': 'ebass@enron.com', 'from': 0, 'to': 1, 'total': 1},
 {'Contact': 'dfranklin@hanovermeasurement.com, jtgoertz@hotmail.com, \r\n\tdaphneco64@bigplanet.com, shanna.husser@enron.com, \r\n\teric.bass@enron.com',
  'from': 0,
  'to': 1,
  'total': 1},
 {'Contact': 'daphneco64@bigplanet.com', 'from': 1, 'to': 0, 'total': 1}]